## K-mean Model practice
Solution: use K-mean model to output
Source: seed_sample_2000_input.csv

In [1]:
from flask import Flask, request, redirect, render_template, Response, send_file, send_from_directory
from flask_wtf import FlaskForm
from wtforms import SelectMultipleField
from flask_bootstrap import Bootstrap
from os.path import join, dirname, realpath
import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer  # add CountVectorizer for NMF
from sklearn.cluster import KMeans
from datetime import datetime
import io, os
import numpy as np

Import library for stopwords

In [2]:
from sklearn import decomposition
import matplotlib.pyplot as plt
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shouqiangye/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Set DF from csv file
# Source 1: <a href="https://github.com/UNCWellington/AI-tools/blob/main/seed_sample_2000_input.csv?raw=true">seed_sample_2000_input.csv</a>  only 2000 rows  

# Source 2: <a href="https://github.com/UNCWellington/AI-tools/blob/main/Sample-for-SC.csv?raw=true">Sample-for-SC.csv   </a> nearly 10,000 rows
Note: we shoud add ?raw=true at the end of github link address

In [4]:
# url = 'https://github.com/UNCWellington/AI-tools/blob/main/seed_sample_2000_input.csv?raw=true'
url = 'https://github.com/UNCWellington/AI-tools/blob/main/Sample-for-SC.csv?raw=true'
# df = pd.read_csv(url,index_col=0)
df = pd.read_csv(url, keep_default_na=False)

In [5]:
df.head()

,AN,Seed,TAB
0,1783,,"""Anywhere but here"": Querying spatial stigma a..."
1,4672,,"""Are We Safe Analysts?"" Cisgender Countertrans..."
2,4444,,"""As a Trans Person You Don't Live. You Merely ..."
3,22,,"""At Your Service"": Sexual Harassment of Female..."
4,2808,,"""Bareback"" pornography consumption and safe-se..."


set stop words

In [6]:
stemmer = nltk.stem.SnowballStemmer('english')
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shouqiangye/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


set analyse column

In [7]:
documents = df['TAB'].values.astype("U")

In [8]:
#documents

In [9]:
vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 3))
features = vectorizer.fit_transform(documents)

set k-10, length =1, then fit model

In [10]:
k = 10
length = 1


# why every time has different outcome even with the same parameters?

https://stackoverflow.com/questions/25921762/changes-of-clustering-results-after-each-time-run-in-python-scikit-learn

Explanation:
    When using k-means, you want to set the random_state parameter in KMeans (see the documentation). Set this to either an int or a RandomState instance.

# km = KMeans(n_clusters=number_of_k, init='k-means++', max_iter=100, n_init=1, verbose=0, random_state=3425)

This is important because k-means is not a deterministic algorithm. 
It usually starts with some randomized initialization procedure, 
and this randomness means that different runs will start at different points. 
Seeding the pseudo-random number generator ensures that this randomness will always be the same for identical seeds.

In [11]:
# model = KMeans(n_clusters=k, init='k-means++', max_iter=int(length), n_init=1)
model = KMeans(n_clusters=k, init='k-means++', max_iter=int(length), n_init=1, verbose=0, random_state=3425)
# recommended by Adam to fix the bug that different outcome every time
# model = KMeans(n_clusters=k, init='k-means++', n_init=10, max_iter=300, tol=0.0001, verbose=0, random_state=42, copy_x=True, algorithm='auto')
# model = KMeans(n_clusters=k, init='k-means++', n_init=1, max_iter=300, tol=0.0001, verbose=0, random_state=42, copy_x=True, algorithm='auto')
model.fit(features)

KMeans(max_iter=1, n_clusters=10, n_init=1, random_state=3425)

In [12]:
model.labels_

array([8, 6, 6, ..., 5, 7, 1], dtype=int32)

In [13]:
df['Cluster'] = model.labels_
df2 = pd.DataFrame()
clusters = df.groupby('Cluster')

In [14]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(k):
    subset_df = df[df.Cluster == i].shape[0]

    top_ten_words = [terms[ind] for ind in order_centroids[i, :10]]
        # df2['top_ten_words'] = [terms[ind] for ind in order_centroids[i, :10]]
        # print("Cluster {}: {}".format(i, '; '.join(top_ten_words)))

    data = {'Cluster Number': i, 'Number of Items': subset_df,
                'Top Cluster Terms': top_ten_words}

    df2 = df2.append(data, ignore_index=True)

In [15]:
df2

,Cluster Number,Number of Items,Top Cluster Terms
0,0.0,423.0,"[species, gene, asexual, malaria, parasite, ge..."
1,1.0,1706.0,"[gender, identity, self, female, male, sex, ge..."
2,2.0,874.0,"[msm, hiv, men, prep, sex, among, art, men sex..."
3,3.0,1476.0,"[sexual, women, sexual minority, minority, dep..."
4,4.0,601.0,"[use, sexual, 95, identity, ci, among, alcohol..."
5,5.0,1470.0,"[hiv, men, prep, sex, risk, sexual, among, msm..."
6,6.0,1515.0,"[health, transgender, mental, mental health, s..."
7,7.0,1112.0,"[sexual, school, gender, students, orientation..."
8,8.0,340.0,"[covid, covid 19, health, 19, pandemic, social..."
9,9.0,577.0,"[attitudes, toward, attitudes toward, gay, stu..."
